In [1]:

import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
#readdataset
df1 = pd.read_excel('ar-2010-2014.xlsx')
df2 = pd.read_excel('ar-2015-2016.xlsx')
#check dataset1 & 2 shape
print(df1.shape)
print(df2.shape)
#combine dataset
dfall = pd.concat([df1.reset_index(drop = True), df2.reset_index(drop = True)], axis=0)
#check combinedataset shape
print(dfall.shape)
#identified data column
dfall.info()

(337109, 95)
(158519, 95)
(495628, 95)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 495628 entries, 0 to 158518
Data columns (total 95 columns):
 #   Column                                                                    Non-Null Count   Dtype  
---  ------                                                                    --------------   -----  
 0   Patient Age at Treatment                                                  495628 non-null  object 
 1   Date patient started trying to become pregnant OR date of last pregnancy  12891 non-null   float64
 2   Total Number of Previous cycles, Both IVF and DI                          495628 non-null  object 
 3   Total Number of Previous treatments, Both IVF and DI at clinic            495628 non-null  object 
 4   Total Number of Previous IVF cycles                                       495628 non-null  object 
 5   Total Number of Previous DI cycles                                        495628 non-null  object 
 6   Total number 

In [3]:
#DataSplit Tabular

def split_train_test(x, y, TEST_SIZE):
    # Do not forget to stratify if classification
    x_train, x_test, y_train, y_test = train_test_split(x,
                                           y,
                                           test_size=TEST_SIZE,
                                           random_state=123, shuffle= True, stratify=y )

    return x_train, x_test, y_train, y_test

def split_data(data_input, data_ouput, return_file=False, TEST_SIZE=0.2):

    x_train, x_test, y_train, y_test = split_train_test(
            data_input,
            data_ouput,
            TEST_SIZE)

    x_train, x_valid, y_train, y_valid = split_train_test(
            x_train,
            y_train,
            TEST_SIZE)

    joblib.dump(x_train, "x_train.pkl")
    joblib.dump(y_train, "y_train.pkl")
    joblib.dump(x_valid, "x_valid.pkl")
    joblib.dump(y_valid, "y_valid.pkl")
    joblib.dump(x_test, "x_test.pkl")
    joblib.dump(y_test, "y_test.pkl")

    if return_file:
        return x_train, y_train, \
            x_valid, y_valid, \
            x_test, y_test
    
def split_input_output(dataset,
                       target_column,
                       save_file = True,
                       return_file = True):
    
    output_df = dataset[target_column]
    input_df = dataset.drop([target_column],
                            axis = 1)
    
    if save_file:
        joblib.dump(output_df, "output_df.pkl")
        joblib.dump(input_df, "input_df.pkl")
    
    if return_file:
        return output_df, input_df
    
    
TARGET_COLUMN = "Live Birth Occurrence"
INDEX_COLUMN = "Id"
TEST_SIZE = 0.2


output_df, input_df = split_input_output(
                            dfall,
                            TARGET_COLUMN)

output_df.fillna(0,inplace = True)



x_train, x_test, y_train, y_test = train_test_split(input_df,
                                           output_df,
                                           test_size=TEST_SIZE,
                                           random_state=123, shuffle= True,stratify = output_df )


In [4]:
#DATA PREPROCESSING

#EDA & FEATURE ENGINEERING

#DROP Unrelated Feature

#As the case is not a time series , we see Date as a unnecessary Features as well
#we drop number of life birth besides live birth occurances to avoid any data leak

x_train = x_train.drop(["Number of Live Births","Specific treatment type","Main Reason for Producing Embroys Storing Eggs","Heart One Delivery Date","Heart Two Delivery Date","Heart Three Delivery Date","Heart Four Delivery Date","Date patient started trying to become pregnant OR date of last pregnancy","Year of Treatment","Date of Egg Thawing","Date of Egg Mixing","Date of Embryo Thawing","Date of Embryo Transfer","Date of Egg Collection"], axis=1)

x_test = x_test.drop(["Number of Live Births","Specific treatment type","Main Reason for Producing Embroys Storing Eggs","Heart One Delivery Date","Heart Two Delivery Date","Heart Three Delivery Date","Heart Four Delivery Date","Date patient started trying to become pregnant OR date of last pregnancy","Year of Treatment","Date of Egg Thawing","Date of Egg Mixing","Date of Embryo Thawing","Date of Embryo Transfer","Date of Egg Collection"], axis=1)

#Check Patient Age at Tretment Data Training
print(x_train["Patient Age at Treatment"].unique())
print("Outlier 999 is identified in Data Training")
print("before outlier Imputation")

checkage = x_train.groupby(["Patient Age at Treatment"])[['Total number of IVF pregnancies','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Total number of IVF pregnancies':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)

print(checkage)
#we've found outlier in data training and we will impute with the nearest categories

x_train["Patient Age at Treatment"] = np.where(x_train["Patient Age at Treatment"] == "999","45-50",x_train["Patient Age at Treatment"])

print("After outlier Imputation")

checkage = x_train.groupby(["Patient Age at Treatment"])[['Total number of IVF pregnancies','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Total number of IVF pregnancies':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)

print(checkage)

#Check Patient Age at Tretment Data Testing
print(x_test["Patient Age at Treatment"].unique())
print("Outlier 999 is identified in Data testing")
print("before outlier Imputation")

checkage = x_test.groupby(["Patient Age at Treatment"])[['Total number of IVF pregnancies','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Total number of IVF pregnancies':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)

print(checkage)
#we've found outlier in data testing and we will impute with nearest categories

x_test["Patient Age at Treatment"] = np.where(x_test["Patient Age at Treatment"] == "999","45-50",x_test["Patient Age at Treatment"])

print("After outlier Imputation")

checkage = x_test.groupby(["Patient Age at Treatment"])[['Total number of IVF pregnancies','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Total number of IVF pregnancies':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)

print(checkage)


#Check Total Number Features"
#Some columns are terminologically correlated, instead of using Total Both IVF and DI, we will used the total each IVF and DI

x_train = x_train.drop(["Total Number of Previous cycles, Both IVF and DI","Total Number of Previous treatments, Both IVF and DI at clinic","Total number of previous pregnancies, Both IVF and DI","Total number of live births - conceived through IVF or DI"], axis=1)

x_test = x_test.drop(["Total Number of Previous cycles, Both IVF and DI","Total Number of Previous treatments, Both IVF and DI at clinic","Total number of previous pregnancies, Both IVF and DI","Total number of live births - conceived through IVF or DI"], axis=1)

print(x_train["Total Number of Previous IVF cycles"].unique())

print("Outlier  is identified in Data training")
print("before outlier Imputation")

checkage = x_train.groupby(["Total Number of Previous IVF cycles"])[['Patient Age at Treatment','Total Number of Previous DI cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous DI cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

print("after  outlier Imputation")
x_train["Total Number of Previous IVF cycles"] = np.where(x_train["Total Number of Previous IVF cycles"] == ">=5","5",x_train["Total Number of Previous IVF cycles"])
checkage = x_train.groupby(["Total Number of Previous IVF cycles"])[['Patient Age at Treatment','Total Number of Previous DI cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous DI cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

#we've found outlier in data training and we will impute with nearest categories

x_test["Total Number of Previous IVF cycles"].unique()

print("Outlier  is identified in Data testing")
print("before outlier Imputation")

checkage = x_test.groupby(["Total Number of Previous IVF cycles"])[['Patient Age at Treatment','Total Number of Previous DI cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous DI cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

print("after  outlier Imputation")
x_test["Total Number of Previous IVF cycles"] = np.where(x_test["Total Number of Previous IVF cycles"] == ">=5","5",x_test["Total Number of Previous IVF cycles"])
checkage = x_test.groupby(["Total Number of Previous IVF cycles"])[['Patient Age at Treatment','Total Number of Previous DI cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous DI cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

#we've found outlier in data training and we will impute with nearest categories

x_train["Total Number of Previous DI cycles"].unique()

print("Outlier  is identified in Data training")
print("before outlier Imputation")

checkage = x_train.groupby(["Total Number of Previous DI cycles"])[['Patient Age at Treatment','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

print("after  outlier Imputation")
x_train["Total Number of Previous DI cycles"] = np.where(x_train["Total Number of Previous DI cycles"] == ">=5","5",x_train["Total Number of Previous DI cycles"])
checkage = x_train.groupby(["Total Number of Previous DI cycles"])[['Patient Age at Treatment','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

#we've found outlier in data training and we will impute with nearest categories

x_test["Total Number of Previous DI cycles"].unique()

print("Outlier  is identified in Data testing")
print("before outlier Imputation")

checkage = x_test.groupby(["Total Number of Previous DI cycles"])[['Patient Age at Treatment','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

print("after  outlier Imputation")
x_test["Total Number of Previous DI cycles"] = np.where(x_test["Total Number of Previous DI cycles"] == ">=5","5",x_test["Total Number of Previous DI cycles"])
checkage = x_test.groupby(["Total Number of Previous DI cycles"])[['Patient Age at Treatment','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

#we've found outlier in data training and we will impute with nearest categories

x_train["Total number of IVF pregnancies"].unique()

print("Outlier  is identified in Data training")
print("before outlier Imputation")

checkage = x_train.groupby(["Total number of IVF pregnancies"])[['Patient Age at Treatment','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

print("after  outlier Imputation")
x_train["Total number of IVF pregnancies"] = np.where(x_train["Total number of IVF pregnancies"] == ">=5","5",x_train["Total number of IVF pregnancies"])
checkage = x_train.groupby(["Total number of IVF pregnancies"])[['Patient Age at Treatment','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

#we've found outlier in data training and we will impute with nearest categories

x_test["Total number of IVF pregnancies"].unique()

print("Outlier  is identified in Data testing")
print("before outlier Imputation")

checkage = x_test.groupby(["Total number of IVF pregnancies"])[['Patient Age at Treatment','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

print("after  outlier Imputation")
x_test["Total number of IVF pregnancies"] = np.where(x_test["Total number of IVF pregnancies"] == ">=5","5",x_test["Total number of IVF pregnancies"])
checkage = x_test.groupby(["Total number of IVF pregnancies"])[['Patient Age at Treatment','Total Number of Previous IVF cycles']].count()
checkage.rename(columns = {'Patient Age at Treatment':'Total', 'Total Number of Previous IVF cycles':'Percentage'}, inplace = True)
checkage['percent'] = (100*checkage["Total"]/checkage["Total"].sum()).sort_values(ascending=False)
print(checkage)

#we've found outlier in data training and we will impute with nearest categories


#Typecasting Total Number from Object to Numerical Float

x_train['Total Number of Previous IVF cycles'] = x_train['Total Number of Previous IVF cycles'].astype('float')
x_train['Total Number of Previous DI cycles'] = x_train['Total Number of Previous DI cycles'].astype('float')
x_train['Total number of IVF pregnancies'] = x_train['Total number of IVF pregnancies'].astype('float')
x_train['Total number of DI pregnancies'] = x_train['Total number of DI pregnancies'].astype('float')
x_train['Total number of live births - conceived through IVF'] = x_train['Total number of live births - conceived through IVF'].astype('float')
x_train['Total number of live births - conceived through DI'] = x_train['Total number of live births - conceived through DI'].astype('float')

x_test['Total Number of Previous IVF cycles'] = x_test['Total Number of Previous IVF cycles'].astype('float')
x_test['Total Number of Previous DI cycles'] = x_test['Total Number of Previous DI cycles'].astype('float')
x_test['Total number of IVF pregnancies'] = x_test['Total number of IVF pregnancies'].astype('float')
x_test['Total number of DI pregnancies'] = x_test['Total number of DI pregnancies'].astype('float')
x_test['Total number of live births - conceived through IVF'] = x_test['Total number of live births - conceived through IVF'].astype('float')
x_test['Total number of live births - conceived through DI'] = x_test['Total number of live births - conceived through DI'].astype('float')


#From total numbers we can create new features which can summarize  into fewer features

x_train["livingbirth_rate_IVFprev"] = x_train["Total number of live births - conceived through IVF"]/x_train['Total Number of Previous IVF cycles']
x_train["livingbirth_rate_DIprev"] = x_train["Total number of live births - conceived through DI"]/x_train['Total Number of Previous DI cycles']

x_test["livingbirth_rate_IVFprev"] = x_test["Total number of live births - conceived through IVF"]/x_test['Total Number of Previous IVF cycles']
x_test["livingbirth_rate_DIprev"] = x_test["Total number of live births - conceived through DI"]/x_test['Total Number of Previous DI cycles']


x_train["livingbirth_rate_IVFprev_tag"] = np.where(x_train["livingbirth_rate_IVFprev"]<= 0.25,"<25 IVF_liverate",np.where((x_train["livingbirth_rate_IVFprev"]> 0.25)&((x_train["livingbirth_rate_IVFprev"]<= 0.5)),"25-50 IVF_liverate",np.where((x_train["livingbirth_rate_IVFprev"]> 0.5)&(x_train["livingbirth_rate_IVFprev"]<= 0.75),"50-75 IVF_liverate",np.where((x_train["livingbirth_rate_IVFprev"]> 0.75),">75 IVF_liverate","norecordIVF"))))
x_test["livingbirth_rate_IVFprev_tag"] = np.where(x_test["livingbirth_rate_IVFprev"]<= 0.25,"<25 IVF_liverate",np.where((x_test["livingbirth_rate_IVFprev"]> 0.25)&((x_test["livingbirth_rate_IVFprev"]<= 0.5)),"25-50 IVF_liverate",np.where((x_test["livingbirth_rate_IVFprev"]> 0.5)&(x_test["livingbirth_rate_IVFprev"]<= 0.75),"50-75 IVF_liverate",np.where((x_test["livingbirth_rate_IVFprev"]> 0.75),">75 IVF_liverate","norecordIVF"))))



x_train["livingbirth_rate_DIprev_tag"] = np.where(x_train["livingbirth_rate_DIprev"]<= 0.25,"<25 DI_liverate",np.where((x_train["livingbirth_rate_DIprev"]> 0.25)&((x_train["livingbirth_rate_DIprev"]<= 0.5)),"25-50 DI_liverate",np.where((x_train["livingbirth_rate_DIprev"]> 0.5)&(x_train["livingbirth_rate_DIprev"]<= 0.75),"50-75 DI_liverate",np.where((x_train["livingbirth_rate_DIprev"]> 0.75),">75 DI_liverate","norecordDI"))))
x_test["livingbirth_rate_DIprev_tag"] = np.where(x_test["livingbirth_rate_DIprev"]<= 0.25,"<25 DI_liverate",np.where((x_test["livingbirth_rate_DIprev"]> 0.25)&((x_test["livingbirth_rate_DIprev"]<= 0.5)),"25-50 DI_liverate",np.where((x_test["livingbirth_rate_DIprev"]> 0.5)&(x_test["livingbirth_rate_DIprev"]<= 0.75),"50-75 DI_liverate",np.where((x_test["livingbirth_rate_DIprev"]> 0.75),">75 DI_liverate","norecordDI"))))


#We dont take into count pregnancy rate as it was terminologically Correlated  with Living Birth Rate

#We drop redundant columns 
x_train = x_train.drop(["livingbirth_rate_IVFprev","livingbirth_rate_DIprev","Total number of IVF pregnancies","Total number of DI pregnancies","Total number of live births - conceived through IVF","Total number of live births - conceived through DI","Total Number of Previous IVF cycles","Total Number of Previous DI cycles"], axis=1)

x_test = x_test.drop(["livingbirth_rate_IVFprev","livingbirth_rate_DIprev","Total number of IVF pregnancies","Total number of DI pregnancies","Total number of live births - conceived through IVF","Total number of live births - conceived through DI","Total Number of Previous IVF cycles","Total Number of Previous DI cycles"], axis=1)


#Check % Missing Value data to eliminate Features with substantial number of missing value [ >10%]
total = x_train.isna().sum().sort_values(ascending=False)
percent = (x_train.isna().sum()/x_train.isna().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
removedmissingdatacolumn = missing_data.loc[missing_data["Percent"]>0.1].reset_index()['index'].to_list()
x_train = x_train.drop(removedmissingdatacolumn, axis = 1)
x_test = x_test.drop(removedmissingdatacolumn, axis = 1)

#Droprow with minor qth of missing value
combinetrain = pd.concat([x_train, y_train], axis=1)
combinetest= pd.concat([x_test, y_test], axis=1)
combinetrain_nona = combinetrain.dropna(axis=0)
combinetest_nona = combinetest.dropna(axis=0)
print(combinetrain.shape)
print(combinetrain_nona.shape)
print(combinetest.shape)
print(combinetest_nona.shape)

y_train = combinetrain_nona['Live Birth Occurrence']
x_train = combinetrain_nona.drop('Live Birth Occurrence', axis = 1)
y_test = combinetest_nona['Live Birth Occurrence']
x_test = combinetest_nona.drop('Live Birth Occurrence', axis = 1)

total = x_train.isna().sum().sort_values(ascending=False)
percent = (x_train.isna().sum()/x_train.isna().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

missing_data

#DROP Column with one kind of value

x_train = x_train.drop(['Type of treatment - IVF or DI','Cause of Infertility - Female Factors'], axis = 1)
x_test = x_test.drop(['Type of treatment - IVF or DI','Cause of Infertility - Female Factors'], axis = 1)

#Change Object Features to Numerical

# Convert categories into numerical to increase the speed of Model

x_train["Patient Age at Treatment"] = np.where(x_train["Patient Age at Treatment"] == '18 - 34',0,np.where(x_train["Patient Age at Treatment"] == '35-37',1,np.where(x_train["Patient Age at Treatment"] == '38-39',2,np.where(x_train["Patient Age at Treatment"] == '40-42',3,np.where(x_train["Patient Age at Treatment"] == '43-44',4,np.where(x_train["Patient Age at Treatment"] == '45-50',5,6))))))
x_train["Patient Age at Treatment"] = x_train["Patient Age at Treatment"].astype('float')

x_train['Fresh Eggs Collected'] = np.where(x_train['Fresh Eggs Collected'] == '> 50','50',x_train['Fresh Eggs Collected'])
x_train['Fresh Eggs Collected'] = x_train['Fresh Eggs Collected'].astype('float')

x_train['Fresh Eggs Stored'] = np.where(x_train['Fresh Eggs Stored'] == '> 50','50',x_train['Fresh Eggs Stored'])
x_train['Fresh Eggs Stored'] = x_train['Fresh Eggs Stored'].astype('float')

x_train['Fresh Eggs Stored'] = np.where(x_train['Fresh Eggs Stored'] == '> 50','50',x_train['Fresh Eggs Stored'])
x_train['Fresh Eggs Stored'] = x_train['Fresh Eggs Stored'].astype('float')

x_train['Total Eggs Mixed'] = np.where(x_train['Total Eggs Mixed'] == '> 50','50',x_train['Total Eggs Mixed'])
x_train['Total Eggs Mixed'] = x_train['Total Eggs Mixed'].astype('float')

x_train['Eggs Mixed With Partner Sperm'] = np.where(x_train['Eggs Mixed With Partner Sperm'] == '> 50','50',x_train['Eggs Mixed With Partner Sperm'])
x_train['Eggs Mixed With Partner Sperm'] = x_train['Eggs Mixed With Partner Sperm'].astype('float')

x_train['Eggs Mixed With Donor sperm'] = np.where(x_train['Eggs Mixed With Donor sperm'] == '> 50','50',x_train['Eggs Mixed With Donor sperm'])
x_train['Eggs Mixed With Donor sperm'] = x_train['Eggs Mixed With Donor sperm'].astype('float')

x_train['Total Embryos Created'] = np.where(x_train['Total Embryos Created'] == '> 50','50',x_train['Total Embryos Created'])
x_train['Total Embryos Created'] = x_train['Total Embryos Created'].astype('float')

x_train['Eggs Micro-injected'] = np.where(x_train['Eggs Micro-injected'] == '> 50','50',x_train['Eggs Micro-injected'])
x_train['Eggs Micro-injected'] = x_train['Eggs Micro-injected'].astype('float')

x_train['Embryos Stored For Use By Patient'] = np.where(x_train['Embryos Stored For Use By Patient'] == '> 50','50',x_train['Embryos Stored For Use By Patient'])
x_train['Embryos Stored For Use By Patient'] = x_train['Embryos Stored For Use By Patient'].astype('float')

x_train['Embryos (from Eggs Micro-injected) Stored For Use By Patient'] = np.where(x_train['Embryos (from Eggs Micro-injected) Stored For Use By Patient'] == '> 50','50',x_train['Embryos (from Eggs Micro-injected) Stored For Use By Patient'])
x_train['Embryos (from Eggs Micro-injected) Stored For Use By Patient'] = x_train['Embryos (from Eggs Micro-injected) Stored For Use By Patient'].astype('float')


x_test['Fresh Eggs Collected'] = np.where(x_test['Fresh Eggs Collected'] == '> 50','50',x_test['Fresh Eggs Collected'])
x_test['Fresh Eggs Collected'] = x_test['Fresh Eggs Collected'].astype('float')

x_test['Fresh Eggs Stored'] = np.where(x_test['Fresh Eggs Stored'] == '> 50','50',x_test['Fresh Eggs Stored'])
x_test['Fresh Eggs Stored'] = x_test['Fresh Eggs Stored'].astype('float')

x_test['Fresh Eggs Stored'] = np.where(x_test['Fresh Eggs Stored'] == '> 50','50',x_test['Fresh Eggs Stored'])
x_test['Fresh Eggs Stored'] = x_test['Fresh Eggs Stored'].astype('float')

x_test['Total Eggs Mixed'] = np.where(x_test['Total Eggs Mixed'] == '> 50','50',x_test['Total Eggs Mixed'])
x_test['Total Eggs Mixed'] = x_test['Total Eggs Mixed'].astype('float')

x_test['Eggs Mixed With Partner Sperm'] = np.where(x_test['Eggs Mixed With Partner Sperm'] == '> 50','50',x_test['Eggs Mixed With Partner Sperm'])
x_test['Eggs Mixed With Partner Sperm'] = x_test['Eggs Mixed With Partner Sperm'].astype('float')

x_test['Eggs Mixed With Donor sperm'] = np.where(x_test['Eggs Mixed With Donor sperm'] == '> 50','50',x_test['Eggs Mixed With Donor sperm'])
x_test['Eggs Mixed With Donor sperm'] = x_test['Eggs Mixed With Donor sperm'].astype('float')

x_test['Total Embryos Created'] = np.where(x_test['Total Embryos Created'] == '> 50','50',x_test['Total Embryos Created'])
x_test['Total Embryos Created'] = x_test['Total Embryos Created'].astype('float')

x_test['Eggs Micro-injected'] = np.where(x_test['Eggs Micro-injected'] == '> 50','50',x_test['Eggs Micro-injected'])
x_test['Eggs Micro-injected'] =x_test['Eggs Micro-injected'].astype('float')

x_test['Embryos Stored For Use By Patient'] = np.where(x_test['Embryos Stored For Use By Patient'] == '> 50','50',x_test['Embryos Stored For Use By Patient'])
x_test['Embryos Stored For Use By Patient'] = x_test['Embryos Stored For Use By Patient'].astype('float')

x_test['Embryos (from Eggs Micro-injected) Stored For Use By Patient'] = np.where(x_test['Embryos (from Eggs Micro-injected) Stored For Use By Patient'] == '> 50','50',x_test['Embryos (from Eggs Micro-injected) Stored For Use By Patient'])
x_test['Embryos (from Eggs Micro-injected) Stored For Use By Patient'] = x_test['Embryos (from Eggs Micro-injected) Stored For Use By Patient'].astype('float')

x_test["Patient Age at Treatment"] = np.where(x_test["Patient Age at Treatment"] == '18 - 34',0,np.where(x_test["Patient Age at Treatment"] == '35-37',1,np.where(x_test["Patient Age at Treatment"] == '38-39',2,np.where(x_test["Patient Age at Treatment"] == '40-42',3,np.where(x_test["Patient Age at Treatment"] == '43-44',4,np.where(x_test["Patient Age at Treatment"] == '45-50',5,6))))))
x_test["Patient Age at Treatment"] = x_test["Patient Age at Treatment"].astype('float')





['18 - 34' '43-44' '35-37' '40-42' '38-39' '45-50' '999']
Outlier 999 is identified in Data Training
before outlier Imputation
                           Total  Percentage    percent
Patient Age at Treatment                               
18 - 34                   168904      168904  42.598524
35-37                      87810       87810  22.146168
38-39                      57846       57846  14.589082
40-42                      53352       53352  13.455670
43-44                      15583       15583   3.930119
45-50                       8384        8384   2.114491
999                         4623        4623   1.165946
After outlier Imputation
                           Total  Percentage    percent
Patient Age at Treatment                               
18 - 34                   168904      168904  42.598524
35-37                      87810       87810  22.146168
38-39                      57846       57846  14.589082
40-42                      53352       53352  13.455670
43-44   

In [5]:
#Check Data Shape After  EDA & Feature Engineering & Data Preprocessing
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(370619, 43)
(370619,)
(92739, 43)
(92739,)


In [6]:
#Onehot_encoding for object features train  for future selection

from sklearn.preprocessing import LabelEncoder 

cat_cols = [e for e in x_train.columns if x_train[e].dtype == "object"]

# print("Categorical columns are : ",cat_cols,x_train.columns)


for ec in cat_cols:
    dummy_col = None
    dummy_col = pd.get_dummies(x_train[ec])
    x_train = pd.concat([x_train,dummy_col],axis=1)
    x_train = x_train.drop([ec],axis=1)
    
#Onehot_encoding for object features Test

cat_cols = [e for e in x_test.columns if x_test[e].dtype == "object"]

# print("Categorical columns are : ",cat_cols,x_train.columns)


for ec in cat_cols:
    dummy_col = None
    dummy_col = pd.get_dummies(x_test[ec])
    x_test = pd.concat([x_test,dummy_col],axis=1)
    x_test = x_test.drop([ec],axis=1)
    
#Check Data Shape after onehot  encoding
print(x_train.shape)
print(x_test.shape)

combinetrain = pd.concat([x_train, y_train], axis=1)

(370619, 55)
(92739, 55)


In [7]:
#ANOTHER DEEP EDA [Check distribution of Probability for each Age Grouping]

print('The younger you are, the higher the probability of Live birth occurance ')

combinetrain[['Patient Age at Treatment', "Live Birth Occurrence"]].groupby(['Patient Age at Treatment'], as_index=False).mean().sort_values(by='Live Birth Occurrence', ascending=False)

The younger you are, the higher the probability of Live birth occurance 


,Patient Age at Treatment,Live Birth Occurrence
0,0.0,0.308679
1,1.0,0.267805
2,2.0,0.206959
3,3.0,0.148533
4,4.0,0.111513
5,5.0,0.105527


In [8]:
#ANOTHER DEEP EDA [Check probability from Training data based on living birth rate of previous IVF]

print(combinetrain[['<25 IVF_liverate', "Live Birth Occurrence"]].groupby(['<25 IVF_liverate'], as_index=False).mean().sort_values(by='Live Birth Occurrence', ascending=False))
print(combinetrain[['>75 IVF_liverate', "Live Birth Occurrence"]].groupby(['>75 IVF_liverate'], as_index=False).mean().sort_values(by='Live Birth Occurrence', ascending=False))

print("if more you have historical of failure of IVF , the smaller of your change having live Birth Occurance")

   <25 IVF_liverate  Live Birth Occurrence
0                 0               0.263007
1                 1               0.232888
   >75 IVF_liverate  Live Birth Occurrence
1                 1               0.295504
0                 0               0.246715
if more you have historical of failure of IVF , the smaller of your change having live Birth Occurance


In [11]:
#FEATURE SELECTION

from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2


# define feature selection based  f_classic
fs = SelectKBest(score_func=f_classif, k=25)
# apply feature selection
fs.fit(x_train, y_train)

vector_namesfs = list(x_train.columns[fs.get_support(indices=True)])


# define feature selection based chi2
ch = SelectKBest(score_func=chi2, k=25)
# apply feature selection
ch.fit(x_train, y_train)

vector_nameschi = list(x_train.columns[ch.get_support(indices=True)])

#combine two methods feature selection

selectedfeature_combine =list(set.intersection(set(vector_nameschi), set(vector_namesfs)))

print(f'Selected feature based on Chi and f_classif resulting {len(selectedfeature_combine)} features')

print(selectedfeature_combine)



Selected feature based on Chi and f_classif resulting 22 features
['Fresh Eggs Collected', 'Number of foetal sacs with fetal pulsation', '<25 IVF_liverate', 'Embryos from Eggs Micro-injected', 'Embryos Stored For Use By Patient', 'Embryos Transfered', 'Cause of Infertility - Ovulatory Disorder', 'Eggs Mixed With Partner Sperm', 'Donor', 'Patient Age at Treatment', '25-50 IVF_liverate', 'Total Eggs Mixed', '>75 IVF_liverate', 'Eggs Mixed With Donor sperm', 'Total Embryos Created', 'Elective Single Embryo Transfer', 'norecordIVF', 'Embryos (from Eggs Micro-injected) Stored For Use By Patient', 'Eggs Micro-injected', 'Cause of Infertility - Male Factor', 'Fresh Eggs Stored', 'Embryos Transfered from Eggs Micro-injected']


In [12]:
#FINAL FEATURES
x_trainfinal = x_train[selectedfeature_combine]
x_trainfinal.head(10)

,Fresh Eggs Collected,Number of foetal sacs with fetal pulsation,<25 IVF_liverate,Embryos from Eggs Micro-injected,Embryos Stored For Use By Patient,Embryos Transfered,Cause of Infertility - Ovulatory Disorder,Eggs Mixed With Partner Sperm,Donor,Donor,Patient Age at Treatment,25-50 IVF_liverate,Total Eggs Mixed,>75 IVF_liverate,Eggs Mixed With Donor sperm,Total Embryos Created,Elective Single Embryo Transfer,norecordIVF,Embryos (from Eggs Micro-injected) Stored For Use By Patient,Eggs Micro-injected,Cause of Infertility - Male Factor,Fresh Eggs Stored,Embryos Transfered from Eggs Micro-injected
99222,0.0,0,1,0.0,0.0,1.0,0,0.0,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
30656,0.0,0,1,0.0,0.0,1.0,0,0.0,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
68274,0.0,0,1,0.0,0.0,1.0,1,0.0,0,0,4.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,1,0.0,0.0
320586,8.0,2,0,6.0,0.0,2.0,0,7.0,0,0,0.0,0,7.0,0,0.0,6.0,0.0,1,0.0,7.0,1,0.0,2.0
211762,0.0,0,0,2.0,0.0,2.0,1,3.0,1,0,0.0,0,3.0,0,0.0,2.0,0.0,1,0.0,3.0,0,0.0,2.0
20267,15.0,1,1,0.0,4.0,1.0,1,15.0,0,0,0.0,0,15.0,0,0.0,11.0,1.0,0,0.0,0.0,0,0.0,0.0
1552,9.0,1,0,6.0,0.0,1.0,0,8.0,0,0,1.0,0,8.0,0,0.0,6.0,0.0,1,0.0,8.0,1,0.0,1.0
179539,0.0,0,1,0.0,0.0,2.0,0,0.0,1,1,3.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
264500,0.0,1,0,0.0,1.0,1.0,1,20.0,1,0,0.0,0,20.0,0,0.0,3.0,1.0,1,0.0,0.0,0,0.0,0.0
78814,0.0,0,1,0.0,0.0,1.0,0,0.0,0,0,1.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
